# Lab | Web Scraping

Welcome to the "Books to Scrape" Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the data available on Books to Scrape, an online platform showcasing a wide variety of books. As data analyst, you have been tasked with scraping a specific subset of book data from Books to Scrape to assist publishing companies in understanding the landscape of highly-rated books across different genres. Your insights will help shape future book marketing strategies and publishing decisions.

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The publishing industry, much like others, utilizes data analytics to understand market trends, reader preferences, and the performance of books based on factors such as genre, author, and ratings. Books to Scrape serves as a rich source of such data, offering detailed information about a diverse range of books, making it an ideal platform for extracting insights to aid in informed decision-making within the literary world.

**Task**

Your task is to create a Python script using BeautifulSoup and pandas to scrape Books to Scrape book data, focusing on book ratings and genres. The script should be able to filter books with ratings above a certain threshold and in specific genres. Additionally, the script should structure the scraped data in a tabular format using pandas for further analysis.

**Expected Outcome**

A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`. The function should scrape book data from the "Books to Scrape" website and return a `pandas` DataFrame with the following columns:

**Expected Outcome**

- A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`.
- The function should return a DataFrame with the following columns:
  - **UPC**: The Universal Product Code (UPC) of the book.
  - **Title**: The title of the book.
  - **Price (£)**: The price of the book in pounds.
  - **Rating**: The rating of the book (1-5 stars).
  - **Genre**: The genre of the book.
  - **Availability**: Whether the book is in stock or not.
  - **Description**: A brief description or product description of the book (if available).
  
You will execute this script to scrape data for books with a minimum rating of `4.0 and above` and a maximum price of `£20`. 

Remember to experiment with different ratings and prices to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [Books to Scrape](https://books.toscrape.com/)


**Hint**

Your first mission is to familiarize yourself with the **Books to Scrape** website. Navigate to [Books to Scrape](http://books.toscrape.com/) and explore the available books to understand their layout and structure. 

Next, think about how you can set parameters for your data extraction:

- **Minimum Rating**: Focus on books with a rating of 4.0 and above.
- **Maximum Price**: Filter for books priced up to £20.

After reviewing the site, you can construct a plan for scraping relevant data. Pay attention to the details displayed for each book, including the title, price, rating, and availability. This will help you identify the correct HTML elements to target with your scraping script.

Make sure to build your scraping URL and logic based on the patterns you observe in the HTML structure of the book listings!


---

**Best of luck! Immerse yourself in the world of books, and may the data be with you!**

**Important Note**:

In the fast-changing online world, websites often update and change their structures. When you try this lab, the **Books to Scrape** website might differ from what you expect.

If you encounter issues due to these changes, like new rules or obstacles preventing data extraction, don’t worry! Get creative.

You can choose another website that interests you and is suitable for scraping data. Options like Wikipedia, The New York Times, or even library databases are great alternatives. The main goal remains the same: extract useful data and enhance your web scraping skills while exploring a source of information you enjoy. This is your opportunity to practice and adapt to different web environments!

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_books(min_rating, max_price):
    url = "https://books.toscrape.com/catalogue/page-1.html"
    books = []

    while url:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")

        # Extract book entries
        for book in soup.find_all('article', class_='product_pod'):
            # Title
            title = book.h3.a['title']
            # Price
            price = float(book.find('p', class_='price_color').text[1:])  # Remove £ sign and convert to float
            # Rating (convert text rating to number)
            rating_text = book.p['class'][1]
            rating = {'One': 1, 'Two': 2, 'Three': 3, 'Four': 4, 'Five': 5}[rating_text]
            
            # Filtering
            if rating >= min_rating and price <= max_price:
                # Book URL for more details
                book_url = "https://books.toscrape.com/catalogue/" + book.h3.a['href']
                book_response = requests.get(book_url)
                book_soup = BeautifulSoup(book_response.content, "html.parser")
                
                # UPC
                upc = book_soup.find('table', class_='table table-striped').find_all('tr')[0].td.text
                # Genre
                genre = book_soup.find('ul', class_='breadcrumb').find_all('li')[2].a.text
                # Availability
                availability = book_soup.find('p', class_='instock availability').text.strip()
                # Description
                description = book_soup.find('meta', {'name': 'description'})['content'].strip()

                books.append({
                    'UPC': upc,
                    'Title': title,
                    'Price (£)': price,
                    'Rating': rating,
                    'Genre': genre,
                    'Availability': availability,
                    'Description': description
                })

        # Find the next page URL
        next_button = soup.find('li', class_='next')
        url = "https://books.toscrape.com/catalogue/" + next_button.a['href'] if next_button else None

    # Create a DataFrame from the list of books
    return pd.DataFrame(books)

# Example usage
df = scrape_books(4, 20)
#df.info()
df.head(10)




,UPC,Title,Price (£),Rating,Genre,Availability,Description
0,ce6396b0f23f6ecc,Set Me Free,17.46,5,Young Adult,In stock (19 available),Aaron Ledbetter’s future had been planned out ...
1,6258a1f6a6dcfe50,The Four Agreements: A Practical Guide to Pers...,17.66,5,Spirituality,In stock (18 available),"In The Four Agreements, don Miguel Ruiz reveal..."
2,6be3beb0793a53e7,Sophie's World,15.94,5,Philosophy,In stock (18 available),A page-turning novel that is also an explorati...
3,657fe5ead67a7767,Untitled Collection: Sabbath Poems 2014,14.27,4,Poetry,In stock (16 available),"More than thirty-five years ago, when the weat..."
4,51653ef291ab7ddc,This One Summer,19.49,4,Sequential Art,In stock (16 available),"Every summer, Rose goes with her mom and dad t..."
5,709822d0b5bcb7f4,Thirst,17.27,5,Fiction,In stock (16 available),"On a searing summer Friday, Eddie Chapman has ..."
6,0fa6dceead7ce47a,"Princess Jellyfish 2-in-1 Omnibus, Vol. 01 (Pr...",13.61,5,Sequential Art,In stock (16 available),THE LONG-AWAITED STORY OF FANGIRLS TAKING ON T...
7,0e691eda369f4e09,Princess Between Worlds (Wide-Awake Princess #5),13.34,5,Fantasy,In stock (16 available),Just as Annie and Liam are busy making plans t...
8,889139b8e9c4cb36,"Outcast, Vol. 1: A Darkness Surrounds Him (Out...",15.44,4,Sequential Art,In stock (16 available),NEW HORROR SERIES FROM THE WALKING DEAD CREATO...
9,72f9d5be3472d34e,Mama Tried: Traditional Italian Cooking for th...,14.02,4,Food and Drink,In stock (16 available),Cecilia Granata grew up cooking with her famil...
